In [1]:
from pydrake.geometry import SceneGraph, StartMeshcat
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7003


In [3]:
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.visualization import AddDefaultVisualization
from pydrake.geometry import SceneGraph, StartMeshcat

# Create builder and scene graph
builder = DiagramBuilder()
scene_graph = SceneGraph()
scene_graph.set_name("scene_graph")

# Add plant and scene graph to the builder
plant, scene_graph = AddMultibodyPlantSceneGraph(
    builder, time_step=0.01, scene_graph=scene_graph
)

# Parse the URDF model
parser = Parser(plant)
parser.AddModels("stick_bot_generated_fixed_inertia.urdf")
plant.Finalize()

# Start Meshcat visualization
AddDefaultVisualization(builder=builder, meshcat=meshcat)

# Finalize the plant and build the diagram
diagram = builder.Build()
context = diagram.CreateDefaultContext()

# Initialize and run the simulator
sim = Simulator(diagram, context)
context = sim.get_mutable_context()
plant_context = plant.GetMyContextFromRoot(context)

q0 = plant.GetPositions(plant_context)
q0[6] = 0.2
plant.SetPositions(plant_context, q0)
sim.Initialize()


In [4]:
context.SetTime(0.0)
plant.SetPositions(plant_context, q0)
sim.Initialize()
meshcat.StartRecording()
sim.AdvanceTo(5.0)
meshcat.StopRecording()
meshcat.PublishRecording()